In [2]:
from tifffile import imread, imsave
from skimage.filters import threshold_otsu
from skimage.segmentation import clear_border
from skimage.measure import label, regionprops_table, regionprops
from skimage.morphology import closing, square
from skimage.color import label2rgb
from skimage import img_as_uint
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

import pandas as pd
from scipy import spatial
import numpy as np

In [296]:
data_0 = imread('/data/data_drive/processing/Mike/data.tif')

In [297]:
data_0.shape

(130, 659, 661)

In [300]:
for i in range(data_0.shape[0]):
    imsave('/data/data_drive/processing/Mike/seperated/'+str(i)+'.tif',data_0[i,:,:])

In [3]:
data = imread('/data/data_drive/processing/Mike/data_watershed.tif')

In [34]:
thresh = filters.threshold_otsu(data[10])
bw = closing(data[10] > thresh, square(3))

In [95]:
data.shape

(130, 659, 661)

In [4]:
N_rows = 114 
N_cols = data.shape[0] * 4
df = pd.DataFrame(np.zeros((N_rows, N_cols)))
label_image = label(data[0])
region_current = regionprops_table(label_image, properties=['label','centroid'])
#df = pd.DataFrame(region_current)

In [5]:
im = []
for frame in range(data.shape[0]):
    label_image = label(data[frame])
    image_label_overlay = np.expand_dims(label2rgb(label_image, image=data[frame]), axis=0)
    im.append(img_as_uint(image_label_overlay))
    #fig, ax = plt.subplots(figsize=(10, 6))
    #ax.imshow(image_label_overlay)
im = np.concatenate(im, axis=0)

In [293]:
im.shape

(130, 659, 661, 3)

In [294]:
imsave('/home/ninatubau/Desktop/result.tif',im)

In [155]:
df = pd.DataFrame()
LAB= [] 
for frame in range(data.shape[0]-1): 
    pos = [-1000] * 114 
    dist = [-1000] * 114 
    cent0 = [-1000] * 114 
    cent1 = [-1000] * 114 
    label_image = label(data[frame])
    label_image_next = label(data[frame+1])
    #image_label_overlay = label2rgb(label_image, image=data[frame])
    #fig, ax = plt.subplots(figsize=(10, 6))
    #ax.imshow(image_label_overlay)
    region_current = regionprops(label_image)
    region_next = regionprops(label_image_next)

    LAB.append(len(region_current))
    centroids = [region_next[i].centroid for i in range(len(region_next))]
    tree = spatial.KDTree(centroids)
    
    for i in range(len(region_current)):
        new_point = region_current[i].centroid
        position = tree.query([new_point])[1][0]
        distance = tree.query([new_point])[0][0]
        value = region_next[position].centroid
        pos[i] = position
        dist[i] = distance 
        cent0[i] = value[0]
        cent1[i] = value[1]
    name_0 = 'centroid_x'+str(frame)
    name_1 = 'centroid_y'+str(frame)
    pos_ = 'pos_'+str(frame)
    dist_ = 'dist_'+str(frame)
    
    df[name_0] = cent0
    df[name_1] = cent1
    df[pos_] = pos
    df[dist_] = dist

In [120]:
condition_no_duplicates = (~df.duplicated(['pos_0'],keep=False))
condition_duplicates_small_dist = ((df.duplicated(['pos_0'],keep=False)) & (df['dist_0']<7.5))
indexes = df[~(condition_no_duplicates | condition_duplicates_small_dist)].index

In [166]:
df[1:3]

,centroid_x0,centroid_y0,pos_0,dist_0,centroid_x1,centroid_y1,pos_1,dist_1,centroid_x2,centroid_y2,...,pos_126,dist_126,centroid_x127,centroid_y127,pos_127,dist_127,centroid_x128,centroid_y128,pos_128,dist_128
1,111.531481,309.088889,3,0.654228,106.052174,229.667081,1,1.200509,106.241060,229.557616,...,1,5.106344,98.821963,441.422629,1,0.425825,100.221258,437.40564,1,4.253731
2,116.458967,199.762918,2,0.509141,0.000000,0.000000,2,10.950474,117.806191,199.834455,...,2,3.123419,154.800000,346.600000,4,3.359173,0.000000,0.00000,2,9.356825


In [163]:
for frame in range(int(len(df.columns)/4)):
    centroid_x = 'centroid_x'+str(frame)
    centroid_y = 'centroid_y'+str(frame)
    position = 'pos_'+str(frame)
    distance = 'dist_'+str(frame)
    condition_no_duplicates = (~df.duplicated([position],keep=False))
    condition_duplicates_small_dist = ((df.duplicated([position],keep=False)) & (df[distance]<7.5))
    indexes = df[~(condition_no_duplicates | condition_duplicates_small_dist)].index
    for ind in indexes:
        df.at[ind, centroid_x] = 0 
        df.at[ind, centroid_y] = 0 


In [165]:
export_csv = df.to_csv ('/home/ninatubau/Desktop/dataframe.csv', index = None, header=True)

In [6]:
point = df.iloc[[4]].as_matrix()
even = np.arange(0,258,2)
odd = np.arange(1,258,2)
x_point = []
y_point = []
for i in even: 
    x_point.append(point[0,i]) 
for i in odd: 
    y_point.append(point[0,i])   

/home/ninatubau/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
